In [57]:
"""
Refs:
https://gist.github.com/yspkm/f33d59181b7f6f5c8701360995c07418
https://tex.stackexchange.com/questions/112734/is-it-possible-to-install-only-the-required-latex-tool-pdflatex-in-ubuntu
https://github.com/openai/openai-python
"""

'\nRefs:\nhttps://gist.github.com/yspkm/f33d59181b7f6f5c8701360995c07418\nhttps://tex.stackexchange.com/questions/112734/is-it-possible-to-install-only-the-required-latex-tool-pdflatex-in-ubuntu\nhttps://github.com/openai/openai-python\n'

In [58]:
import os
from openai import OpenAI
from dotenv import load_dotenv

def ask_chat_gpt(prompt):
    # Load environment variables from the .env file
    load_dotenv()

    # Access the API key from the environment
    api_key = os.getenv("OPENAI_KEY")

    client = OpenAI(
        # This is the default and can be omitted
        api_key=api_key,
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-3.5-turbo",
    )
    message = chat_completion.choices[0].message.content
    print(message)
    return message

In [59]:
file_path = 'python_code/calculate_hohmann_transfer.py'

# Open the file in read mode and save its contents to a string
with open(file_path, 'r') as file:
    file_contents = file.read()

# Now `file_contents` holds everything in the Python file as a single string
print(file_contents)  # Optional: Print the content to verify


import numpy as np
import matplotlib.pyplot as plt

def calculate_hohmann_transfer(earth_orbit_radius, mars_orbit_radius):
    """
    Calculate the delta-v (velocity change) required for a Hohmann transfer orbit from Earth to Mars.

    Parameters:
    - earth_orbit_radius (float): Radius of Earth's orbit around the Sun in meters.
    - mars_orbit_radius (float): Radius of Mars's orbit around the Sun in meters.

    Returns:
    - delta_v_earth (float): Delta-v required to escape Earth's orbit and enter the transfer orbit.
    - delta_v_mars (float): Delta-v required to enter Mars's orbit from the transfer orbit.
    - total_delta_v (float): Total delta-v for the entire transfer.
    """
    
    # Gravitational constant for the Sun in m^3/s^2
    mu_sun = 1.32712440018e20

    # Calculate orbital velocities of Earth and Mars using circular orbit approximation
    v_earth = np.sqrt(mu_sun / earth_orbit_radius)  # Velocity of Earth in its orbit
    v_mars = np.sqrt(mu_sun / mars_orbit_

In [60]:
start_promt = """
    I would like a LaTeX document template structured as a technical report. This document should be organized into chapters, with each chapter dedicated to a different Python simulation algorithm. The document should include:
    I ONLY want the latex code and no other text. I dont want you to use ''' latex. just the text.
    I want you to make symbols for the variable names. and then describe the symbols with the variable names instead.
    A Front Page with the title and author information.
    Table of Contents for easy navigation.
    Introduction chapter that describes the purpose of the document and provides a general overview of the included Python algorithms.
    Make sure the document is formatet correctly.

    Remember to add:
    usepackage{amsmath}  % For advanced math typesetting
    usepackage{listings} % For syntax-highlighted code without shell-escape

    For each algorithm:

    Create a dedicated chapter.
    Start with a high-level overview of the algorithm.
    Describe the parameters and constants used in the algorithm, as well as the mathematical model.
    Include formulas for calculations in LaTeX style, ensuring they match the Python code logic.
    After the mathematical description, include a chapter at the end where all Python code is displayed in full.
    Ensure each chapter includes a section for any plots or visualizations generated by the algorithm.
    Here is the algoritm: 
    """

In [61]:
total_prompt = start_promt + file_contents
print(total_prompt)


    I would like a LaTeX document template structured as a technical report. This document should be organized into chapters, with each chapter dedicated to a different Python simulation algorithm. The document should include:
    I ONLY want the latex code and no other text. I dont want you to use ''' latex. just the text.
    I want you to make symbols for the variable names. and then describe the symbols with the variable names instead.
    A Front Page with the title and author information.
    Table of Contents for easy navigation.
    Introduction chapter that describes the purpose of the document and provides a general overview of the included Python algorithms.
    Make sure the document is formatet correctly.

    Remember to add:
    usepackage{amsmath}  % For advanced math typesetting
    usepackage{listings} % For syntax-highlighted code without shell-escape

    For each algorithm:

    Create a dedicated chapter.
    Start with a high-level overview of the algorithm.
   

In [62]:
message = ask_chat_gpt(total_prompt)

\documentclass{article}
\usepackage{amsmath} % For advanced math typesetting
\usepackage{listings} % For syntax-highlighted code without shell-escape

\begin{document}

\title{Python Simulation Algorithms Technical Report}
\author{Your Name}
\date{\today}
\maketitle

\tableofcontents

\newpage

\section{Introduction}
This document presents various Python simulation algorithms for scientific calculations.

\chapter{Hohmann Transfer Orbit Algorithm}
The Hohmann transfer orbit algorithm calculates the delta-v required for a Hohmann transfer from Earth to Mars.

\section{Parameters and Constants}
Let $\mu_{\text{sun}}$ be the gravitational constant for the Sun in $m^3/s^2$. \\
Let $v_{\text{earth}}$ be the velocity of Earth in its orbit. \\
Let $v_{\text{mars}}$ be the velocity of Mars in its orbit. \\
Let $R_{\text{earth}}$ be the radius of Earth's orbit around the Sun in meters. \\
Let $R_{\text{mars}}$ be the radius of Mars's orbit around the Sun in meters.

\section{Mathematical Model}

In [63]:
with open("document.tex", "w") as file:
    file.write(message)

In [65]:
import subprocess

subprocess.run(["pdflatex", "document.tex"])

This is pdfTeX, Version 3.141592653-2.6-1.40.24 (TeX Live 2022/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./document.tex
LaTeX2e <2022-11-01> patch level 1
L3 programming layer <2023-01-16>
(/usr/share/texlive/texmf-dist/tex/latex/base/report.cls
Document Class: report 2022/07/02 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsmath.sty
For additional information on amsmath, use the `?' option.
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amstext.sty
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsgen.sty))
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsbsy.sty)
(/usr/share/texlive/texmf-dist/tex/latex/amsmath/amsopn.sty))
(/usr/share/texlive/texmf-dist/tex/latex/listings/listings.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/latex/listings/lstmisc.sty)
(/usr/share/texlive/tex

CompletedProcess(args=['pdflatex', 'document.tex'], returncode=0)